# Self-Hosting an LLM with Ollama and Llama 3.2

This notebook documents the process of setting up and running a self-hosted Large Language Model (LLM) using Ollama with Llama 3.2. It includes the setup of a PostgreSQL database using Docker, the implementation of a LangChain-based backend, and the frontend built with Next.js.

## Choice of Offline LLM API Platform and Model

Chose Ollama with Llama 3.2 for its robust performance and ease of integration. Ollama provides a seamless way to run LLMs locally, ensuring data privacy and reducing latency. Additionally, Llama 3.2 is a lightweight model that can perform well despite the hardware limitations.

## Setting Up the PostgreSQL Database Using Docker

Used Docker Compose to set up a PostgreSQL database. This simplifies the process and ensures consistency across different environments.

### Steps:
1. Create a `docker-compose.yml` file with the following content:
```yaml
version: '3.8'

services:
  db:
    image: postgres:latest
    container_name: postgres
    environment:
      POSTGRES_USER: admin
      POSTGRES_PASSWORD: langchain_123
      POSTGRES_DB: langchaindb
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  postgres_data:
  ```

2. Run the folloing command to start the PostgreSQL database:
```bash
docker-compose up -d
```

## LangChain with Ollama backend
## Implementing the LangChain-Based Backend

Used Flask to create the backend and LangChain to manage the chat history and interactions with the LLM.

### Steps:
#### 1. Create a `ChatBot` class in `backend/utils/chat.py`:
```python
class ChatBot:
    def __init__(self):
        self.llm = llm
        PostgresChatMessageHistory.create_tables(sync_connection, chat_history_table)
        self.chain = prompt | self.llm
        self.chain_with_history = RunnableWithMessageHistory(
            self.chain,
            self.get_session_history,
            input_messages_key="input",
            history_messages_key="history"
        )
```

#### 2. Create a prompt template to include memory handling
```python
prompt = ChatPromptTemplate.from_messages(
        [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]
)
```

#### 3. Create an invoke with history method (for history context)
```python

    def run_with_history(self, input: str, session_id: str) -> str:
        """
        Runs the chatbot with intelligent history handling
        
        Args:
            input (str): User's input message
            session_id (str): Unique identifier for the session
        
        Returns:
            str: AI's response
        """
        try:
            response = self.chain_with_history.invoke(
                {"input": input},
                config={"configurable": {"session_id": session_id}}
            )
            
            return response.content
        
        except Exception as e:
            print(f"Error in run_with_history: {e}")
            # Fallback to simple invoke
            return self.llm.invoke(input)
```

#### 4. Define API endpoints in `app.py`

##### 4.1 Endpoint for sending a message
```python
@app.route('/api/send_message', methods=['POST', 'OPTIONS'])
def send_message():
    """Send a message to the chatbot
    
    Returns:
        Response: JSON response containing the chatbot's response
        
    """
    
    if request.method == 'OPTIONS':
        return _build_cors_preflight_response()

    data = request.get_json()
    prompt = data.get('message')
    session_id = data.get('session_id')
    
    if not prompt:
        return jsonify({"error": "Missing 'prompt' parameter"}), 400

    if not session_id:
        return jsonify({"error": "Missing 'session_id' parameter"}), 400

    if 'session_id' not in session or session['session_id'] != session_id:
        return jsonify({"error": f"Invalid session_id: {session_id}"}), 400

    try:
        logger.info(f"Current Session: {session}")
        response = chatbot.run_with_history(prompt, session_id)
        response_message = response if isinstance(response, str) else str(response)

        return _corsify_actual_response(make_response(jsonify({"message": response_message})))
    except ValueError as e:
        logger.error(f"Error: {e}")
        return jsonify({"error": f"An error has occurred: {e}"}), 500
```

##### 4.2 Endpoint for getting the history for the frontend
```python
@app.route('/api/get_message_history', methods=['POST', 'OPTIONS'])
def get_message_history():
    """Get the message history for the user

    Returns:
        Response: JSON response containing the message history
    """
    
    if request.method == 'OPTIONS':
        return _build_cors_preflight_response()
    
    data = request.json
    session_id = data.get('session_id')
    
    if not session_id:
        return jsonify({"error": "Missing 'session_id' parameter"}), 400
    
    try:
        logger.info(f"Fetching message history for session_id: {session_id}")
        messages = chatbot.get_message_history(session_id)
        if not messages:
            return _corsify_actual_response(make_response(jsonify({"messages": [], "info": "No messages found"}))), 200
        
        return _corsify_actual_response(make_response(jsonify({"messages": messages}))), 200
    except ValueError as e:
        logger.error(f"Error: {e}")
        return jsonify({"error": f"Invalid session_id: {session_id}"}), 500
```


## Choice of Frontend Library and Implementation
Chose Next.js for its powerful features and ease of use. It allows us to build a modern, responsive UI with server-side rendering. Considering that React has a massive amount of support for documentation and from its community - developing React with Next.js will be faster and reliable.

### Steps:
#### 1. Create a `Chat` component in `frontend/components/chat.tsx`:
```tsx
export default function Chat() {
  `Kindly see the chat.tsx file for a complete view`
}
```

#### 2. Within the `Chat` component, create an `apiClient` const
```tsx
  const apiClient = axios.create({
    baseURL: process.env.NEXT_PUBLIC_BACKEND_URL,
    withCredentials: true, // Ensure cookies are included in requests
    headers: {
      "Content-Type": "application/json",
    },
  })
```

#### 3. Handling of sessions
```tsx
  const handleNewSession = async () => {
    try {
      const response = await apiClient.get("api/get_session_id")
      console.log(
        "Received new session ID from backend",
        response.data.session_id
      )
      // Set session_id cookie
      const sessionId = response.data.session_id
      if (sessionId) {
        cookies.set("session_id", sessionId, { path: "/" })
        setMessages([])
      }
    } catch (error) {
      console.error("Error fetching new session ID:", error)
    }
  }

  const handleOldSession = async () => {
    try {
      const response = await apiClient.post("api/get_message_history", {
        session_id: cookies.get("session_id"),
      })
      console.log("Response of request", response)
      setMessages(response.data.messages)
    } catch (error) {
      console.error("Error fetching message history:", error)
    }
  }
```

#### 4. Sending message to the backend
```tsx
  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault()
    if (input.trim()) {
      setMessages((prevMessages) => [
        ...prevMessages,
        { id: prevMessages.length, role: "user", content: input },
      ])
      setInput("")
      setIsReplying(true)

      try {
        const response = await apiClient.post("api/send_message", {
          message: input,
          session_id: cookies.get("session_id"),
        })

        setIsReplying(false)

        setMessages((prevMessages) => [
          ...prevMessages,
          {
            id: prevMessages.length,
            role: "assistant",
            content: response.data.message,
          },
        ])
      } catch (error) {
        console.error(error)
        setMessages((prevMessages) => [
          ...prevMessages,
          {
            id: prevMessages.length,
            role: "assistant",
            content: "I'm sorry, I don't understand that.",
          },
        ])
      }
    }
  }
```

## Note:
``` Encountered an issue with session persistence through cookies - had to look up in stack overflow what the probable reason as to why Flask doesn't retain session cookies in-between requests, it is said that Flask doesn't work well with frontends running at a different port. Since Flask doesn't handle cookies well when the frontend is running in a different port, cookies are manually sent from the backend then the frontend stores it manually.```
```tsx
      const sessionId = response.data.session_id
      if (sessionId) {
        cookies.set("session_id", sessionId, { path: "/" })
        setMessages([])
      }```

## Chatbot tests
Created tests with `pytest` and `pytest-flask` libraries

### Steps:
#### 1. Setup Flask application for testing in `backend/conftest.py`
```python
@pytest.fixture
def app():
    flask_app.config['TESTING'] = True
    return flask_app

@pytest.fixture
def client(app):
    return app.test_client()
```

#### 2. Create classes for each kind of tests in `backend/tests/test_app.py`
```python
class TestSessionEndpoints

class TestMessageHistoryEndpoints

class TestSendMessageEndpoints

